In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.34.0"

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Repositories count: 5
name: central url: https://repo.maven.apache.org/maven2/ release:true update:never snapshot:false update:never 
name: jcenter url: https://jcenter.bintray.com/ release:true update:never snapshot:false update:never 
name: jboss url: https://repository.jboss.org/nexus/content/repositories/releases/ release:true update:never snapshot:false update:never 
name: atlassian url: https://packages.atlassian.com/maven/public release:true update:never snapshot:false update:never 
name: local url: file:///Users/bsorrentino/.m2/repository/ release:true update:never snapshot:true update:always 


In [3]:
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.34.0
Adding dependency dev.langchain4j:langchain4j-open-ai:0.34.0
Solving dependencies
Resolved artifacts count: 26
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core-jdk8/1.0-SNAPSHOT/langgraph4j-core-jdk8-1.0-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/async/async-generator-jdk8/2.0.1/async-generator-jdk8-2.0.1.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-ker

## Initialize Logger

In [4]:
var lm = java.util.logging.LogManager.getLogManager();
lm.checkAccess(); 
try( var file = new java.io.FileInputStream("./logging.properties")) {
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AdaptiveRag");
log.info( "LET'S START" );


2024-09-24 17:14:57 INFO REPL.$JShell$12 do_it$ LET'S START


## Test Issue [#32](https://github.com/bsorrentino/langgraph4j/issues/32)

Issue concerns a problem on `AdaptiveRag` implementation referred to `AnswerGrader` task

In [5]:
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.input.Prompt;
import dev.langchain4j.model.input.structured.StructuredPrompt;
import dev.langchain4j.model.input.structured.StructuredPromptProcessor;
import dev.langchain4j.model.openai.OpenAiChatModel;
import dev.langchain4j.model.output.structured.Description;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.SystemMessage;
import java.time.Duration;
import java.util.function.Function;


public class AnswerGrader implements Function<AnswerGrader.Arguments,AnswerGrader.Score> {

    static final String MODELS[] =  { "gpt-3.5-turbo-0125", "gpt-4o-mini" };

    /**
     * Binary score to assess answer addresses question.
     */
    public static class Score {

        @Description("Answer addresses the question, 'yes' or 'no'")
        public String binaryScore;

        @Override
        public String toString() {
            return "Score: " + binaryScore;
        }
    }

    @StructuredPrompt("""
User question: 

{{question}}

LLM generation: 

{{generation}}
""")
    record Arguments(String question, String generation) {
    }

    interface Service {

        @SystemMessage("""
You are a grader assessing whether an answer addresses and/or resolves a question. 

Give a binary score 'yes' or 'no'. Yes, means that the answer resolves the question otherwise return 'no'
        """)
        Score invoke(String userMessage);
    }

    String openApiKey;

    @Override
    public Score apply(Arguments args) {
        ChatLanguageModel chatLanguageModel = OpenAiChatModel.builder()
                .apiKey( System.getenv("OPENAI_API_KEY")  )
                .modelName( MODELS[1] )
                .timeout(Duration.ofMinutes(2))
                .logRequests(true)
                .logResponses(true)
                .maxRetries(2)
                .temperature(0.0)
                .maxTokens(2000)
                .build();


        Service service = AiServices.create(Service.class, chatLanguageModel);

        Prompt prompt = StructuredPromptProcessor.toPrompt(args);

        log.trace( "prompt: {}", prompt.text() );
        
        return service.invoke(prompt.text());
    }

}


In [17]:

var grader = new AnswerGrader();

var args = new AnswerGrader.Arguments( "What are the four operations ? ", "LLM means Large Language Model" );
grader.apply( args );


2024-09-24 18:37:34 FINEST REPL.$JShell$23$AnswerGrader apply prompt: User question:

What are the four operations ? 

LLM generation:

LLM means Large Language Model



Score: no

In [18]:
var args = new AnswerGrader.Arguments( "What are the four operations", "There are four basic operations: addition, subtraction, multiplication, and division." );
grader.apply( args );


2024-09-24 18:37:35 FINEST REPL.$JShell$23$AnswerGrader apply prompt: User question:

What are the four operations

LLM generation:

There are four basic operations: addition, subtraction, multiplication, and division.



Score: yes